# Loading Dataset

In [ ]:
!pip install rank_bm25 datasets transformers
!pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain pypdf
!pip install --upgrade datasets huggingface_hub fsspec

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.5/305.5 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 4.2 MB/s eta 0:00:

In [ ]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = ''
os.environ['OPENAI_API_KEY'] = ''
import bs4 #extract data from HTML, XML
from langchain import hub # to access pre built components in langchain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma #Chroma for vectorisation
from langchain_core.output_parsers import StrOutputParser # format output
from langchain_core.runnables import RunnablePassthrough #direct pass input, no pipeline processing
from langchain_openai import ChatOpenAI, OpenAIEmbeddings #The LLM model and embedding model
from langchain.document_loaders import PyPDFLoader # to load and parse pdf with metadata
from langchain.prompts import ChatPromptTemplate # contruct template for the model
from langchain_core.runnables import RunnableLambda
import numpy as np

In [ ]:
from datasets import load_dataset
import numpy as np
from rank_bm25 import BM25Okapi
from sklearn.metrics.pairwise import euclidean_distances
from transformers import BertTokenizer

# Load 1% of MS MARCO dataset
#dataset = load_dataset("ms_marco", "v2.1")
dataset = load_dataset("microsoft/ms_marco", "v2.1")

# Define the percentage of data to keep
subset_ratio = 0.0001

# Get the number of samples for each split
train_size = int(len(dataset["train"]) * subset_ratio)
valid_size = int(len(dataset["validation"]) * subset_ratio)
test_size = int(len(dataset["test"]) * subset_ratio)

# Select only 1% of the data
train_data = dataset["train"].shuffle(seed=42).select(range(train_size))
valid_data = dataset["validation"].shuffle(seed=42).select(range(valid_size))
test_data = dataset["test"].shuffle(seed=42).select(range(test_size))

# Initialize tokenizer for follow-up queries
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
print(f"Train samples: {len(train_data)}, Validation samples: {len(valid_data)}, Test samples: {len(test_data)}")

#Train samples: 8087, Validation samples: 1010, Test samples: 1010

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

v2.1/validation-00000-of-00001.parquet:   0%|          | 0.00/210M [00:00<?, ?B/s]

v2.1/train-00000-of-00007.parquet:   0%|          | 0.00/240M [00:00<?, ?B/s]

v2.1/train-00001-of-00007.parquet:   0%|          | 0.00/240M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Train samples: 80, Validation samples: 10, Test samples: 10


In [ ]:
train_data[0]

{'answers': ['Krishnaraj Rai'],
 'passages': {'is_selected': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
  'passage_text': ["Pics/Satej Shinde, Shadab Khan, Bipin Kokate. Aishwarya Rai Bachchan's father Krishnaraj Rai passed away on Saturday at a suburban hospital here due to health issues. He was hospitalised a few weeks ago and was later shifted to the ICU.",
   "Aishwarya's father Krishnaraj Rai's funeral held, celebs pay last respects. Actress Aishwarya Rai Bachchan 's father Krishnaraj Rai passed away recently at a suburban hospital in Mumbai. Rai was hospitalised a few weeks ago and later shifted to the ICU. The funeral took place at Vile Parle Seva Sansthan Shamshan Bhoomi where Aishwarya's father-in-law Amitabh Bachchan also offered prayers.",
   "Aishwarya's father Krishnaraj Rai passed away at the Lilavati Hospital in suburban Bandra, the actress' publicist stated. The funeral took place around 8.30 pm at Vile Parle Seva Sansthan Shamshan Bhoomi where Aishwarya's father-in-law Amitabh Ba

In [ ]:
'''{
  'answers': ['Krishnaraj Rai'],

  'passages': {'is_selected': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
  'passage_text': ["Pics/Satej Shinde, Shadab Khan, Bipin Kokate. Aishwarya Rai Bachchan's father Krishnaraj Rai passed away on Saturday at a suburban hospital here due to health issues. He was hospitalised a few weeks ago and was later shifted to the ICU.", "Aishwarya's father Krishnaraj Rai's funeral held, celebs pay last respects. Actress Aishwarya Rai Bachchan 's father Krishnaraj Rai passed away recently at a suburban hospital in Mumbai. Rai was hospitalised a few weeks ago and later shifted to the ICU. The funeral took place at Vile Parle Seva Sansthan Shamshan Bhoomi where Aishwarya's father-in-law Amitabh Bachchan also offered prayers.", "Aishwarya's father Krishnaraj Rai passed away at the Lilavati Hospital in suburban Bandra, the actress' publicist stated. The funeral took place around 8.30 pm at Vile Parle Seva Sansthan Shamshan Bhoomi where Aishwarya's father-in-law Amitabh Bachchan also offered prayers.", "Aishwarya's father Krishnaraj Rai's funeral: Shah Rukh Khan, Sonali Bendre, and other celebs pay last respect. Aishwarya Rai Bachchan's father Krishnaraj Rai passed away on Saturday at a suburban hospital here due to health issues. He was hospitalised a few weeks ago and was later shifted to the ICU.", 'Aishwarya Rai Bachchan lost her father yesterday. He was ailing since a few days now and breathed his last on Saturday. Shah Rukh Khan, Amitabh Bachchan, Abhishek Bachchan, Sanjay Leela Bhansali and others attended her father’s funeral. Everyone was in a sombre mood but what broke our heart was to see Aishwarya.', 'Aishwarya Rai Family Background, Husband, Daughter, Father and Mother Name. Aishwarya Rai Family Background, Husband, Daughter, Father and Mother Name is written here. Aishwarya Rai IS one of the best actress present in bollywood film industry. She is not only a good actor but also a well know model in bollywood film industry, who came into limelight after winning Miss World Pegeant 1994.', "A Big NOOOO .. Abhishekh Bachchan is the biological father of Aishwarya Rai's child Aaradhya Bachchan. This question arises Because Bachchan baby fails to inherit dad Abhishek's traits. While the nation is busy predicting the new baby Bachchan's looks and features like that of her mom Aishwarya Rai or grandpa Amitabh Bachchan, we have completely forgotten the importance of her dad Abhishek Bachchan.", 'Actor Aishwarya Rai Bachchan’s father Krishnaraj Rai died on Saturday, a report confirms. Pinkvilla confirms the news through Aishwarya’s spokesperson that her father, who was in the intensive care at the Lilavati hospital in Mumbai, passed away due to complications early Saturday. He was an Army biologist. Aishwarya Rai Bachchan with daughter Aaradhya and dad Krishnaraj Rai.', 'Aishwarya Rai Bachchan’s father in hospital, Amitabh Bachchan and Abhishek Bachchan visit. Aishwarya Rai Bachchan is going through a tough time as her father Krishnaraj Rai, who has been in Mumbai’s Lilavati Hospital for almost a month now. This week, hospital sources said he has been shifted to Intensive Care Unit (ICU).', "Aishwarya Rai Bachchan was in tears at her father's funeral. By | Updated: March 19, 2017 2:08 pm. Aishwarya Rai Bachchan-Aaradhya Bachchan return to Mumbai after performing last rites of Krishnaraj Rai - photos. Aishwarya Rai Bachchan and Aaradhya Bachchan spotted at the airport as they head out of the city."],
  'url': ['http://www.mid-day.com/articles/aishwaryas-father-krishnaraj-rai-cremation-amitabh-bachchan-shah-rukh-khan-sonali-bendre-other-celebs-pay-last-respect-vile-parle-bollywood-news/18086172', 'http://timesofindia.indiatimes.com/entertainment/hindi/bollywood/news/aishwarya-rais-father-krishnaraj-rais-funeral-held-celebs-pay-last-respects/articleshow/57711277.cms', 'http://timesofindia.indiatimes.com/entertainment/hindi/bollywood/news/aishwarya-rais-father-krishnaraj-rais-funeral-held-celebs-pay-last-respects/articleshow/57711277.cms', 'http://www.mid-day.com/articles/aishwaryas-father-krishnaraj-rai-cremation-amitabh-bachchan-shah-rukh-khan-sonali-bendre-other-celebs-pay-last-respect-vile-parle-bollywood-news/18086172', 'http://www.bollywoodlife.com/news-gossip/a-teary-eyed-aishwarya-rai-bachchan-at-her-fathers-funeral-will-break-your-heart-view-hq-pics/', 'http://sekho.in/aishwarya-rai-family-background-husband-daughter-father-mother-name/', 'https://www.quora.com/Is-Amitabh-Bachchan-the-biological-father-of-Aishwarya-Rais-child', 'http://www.hindustantimes.com/bollywood/aishwarya-rai-bachchan-s-father-krishnaraj-rai-dies-in-mumbai/story-vhbqFNGAI93llHNRdpVNEO.html', 'http://indianexpress.com/photos/entertainment-gallery/aishwarya-rai-bachchan-father-in-hospital-amitabh-bachchan-and-abhishek-bachchan-visit-4561481/', 'http://www.bollywoodlife.com/news-gossip/a-teary-eyed-aishwarya-rai-bachchan-at-her-fathers-funeral-will-break-your-heart-view-hq-pics/']},

  'query': 'aishwarya rai father',
  'query_id': 14992,
  'query_type': 'PERSON',
  'wellFormedAnswers': []}
  '''

'{\n  \'answers\': [\'Krishnaraj Rai\'],\n\n  \'passages\': {\'is_selected\': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],\n  \'passage_text\': ["Pics/Satej Shinde, Shadab Khan, Bipin Kokate. Aishwarya Rai Bachchan\'s father Krishnaraj Rai passed away on Saturday at a suburban hospital here due to health issues. He was hospitalised a few weeks ago and was later shifted to the ICU.", "Aishwarya\'s father Krishnaraj Rai\'s funeral held, celebs pay last respects. Actress Aishwarya Rai Bachchan \'s father Krishnaraj Rai passed away recently at a suburban hospital in Mumbai. Rai was hospitalised a few weeks ago and later shifted to the ICU. The funeral took place at Vile Parle Seva Sansthan Shamshan Bhoomi where Aishwarya\'s father-in-law Amitabh Bachchan also offered prayers.", "Aishwarya\'s father Krishnaraj Rai passed away at the Lilavati Hospital in suburban Bandra, the actress\' publicist stated. The funeral took place around 8.30 pm at Vile Parle Seva Sansthan Shamshan Bhoomi where Aishwarya\'s fa

#Extracting Questions

In [ ]:
questions = [item['query'] for item in train_data]
questions[0]

'aishwarya rai father'

In [ ]:
len(questions)

80

#Generate Follow-Up Queries

In [ ]:
#post processing LLM output
import ast
def parse_output(output):
    # Remove the triple backticks and any surrounding whitespace
    cleaned_string = output.strip('```python').strip()
    # Safely convert the string to a Python list
    parsed_list = ast.literal_eval(cleaned_string)
    return parsed_list

In [ ]:
# Define the improved prompt template with Few-Shot + Chain-of-Thought (CoT)
prompt_template = """
You are an AI designed to assist in improving Retrieval-Augmented Generation (RAG) by generating follow-up questions.
Your task is to anticipate two relevant follow-up questions that provide additional context or clarity to enhance retrieval accuracy.

To do this, follow these reasoning steps:
Step 1: Break down the user's query into its key components. Identify the main topic, entities, and potential ambiguities.
Step 2: Think of additional details that might be useful for retrieval, such as specific aspects, conditions, or perspectives related to the query.
Step 3: Formulate two insightful follow-up questions that expand on the query to retrieve richer and more relevant information.

### Examples:

User Query: "What are the effects of climate change?"
Reasoning:
1. Climate change has various effects on different domains: environmental, economic, and social.
2. Some key aspects include extreme weather events, rising sea levels, and impact on agriculture.
3. Useful follow-up questions should explore **specific impacts** on different areas.

Follow-up Questions:
["How does climate change specifically impact human health?",
 "What are the economic consequences of climate change on developing countries?"]

---

User Query: "How does deep learning improve image recognition?"
Reasoning:
1. Deep learning improves image recognition by using neural networks, specifically CNNs.
2. Important aspects include feature extraction, data augmentation, and training methods.
3. Good follow-up questions should dive deeper into **technical improvements** and **real-world applications**.

Follow-up Questions:
["What are the advantages of convolutional neural networks (CNNs) in image recognition?",
 "How does transfer learning enhance image recognition models?"]

---

Now, generate follow-up questions ONLY. Do not explain the reasoning.

User Query: "{user_query}"

Output:
"""

# Format the prompt
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini-2024-07-18")
prompt = ChatPromptTemplate.from_template(prompt_template)

follow_up_queries = []
# Process each question
for i in range(len(questions)):
    final_rag_chain = (
        {"user_query": RunnablePassthrough()}  # Pass the user query directly
        | prompt  # Apply the prompt template
        | llm  # Get response from the LLM
        | StrOutputParser()  # Parse the response into a string
        | parse_output # parse string into list  -- Assuming parse_output is defined elsewhere
    )
    # Invoke the chain with the current question
    follow_up_queries.append(final_rag_chain.invoke({"user_query": questions[i]}))

/tmp/ipython-input-68-829991164.py:50: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini-2024-07-18")


In [ ]:
follow_up_queries

[["What is the background and profession of Aishwarya Rai's father?",
  "Are there any notable achievements or contributions of Aishwarya Rai's father in his field?"],
 ["What specific information are you looking for regarding PNC's routing numbers?",
  'Are you interested in routing numbers for personal accounts, business accounts, or both?'],
 ["In what context is the term 'mdatu' used?",
  "Are there any specific cultural or regional references associated with the meaning of 'mdatu'?"],
 ['What specific functions or features are you looking for in a memory card utility?',
  'Are you interested in utilities for a particular operating system or device type?'],
 ['What specific types of medications are known to trigger lupus symptoms?',
  'Are there particular risk factors that increase the likelihood of medication-induced lupus?'],
 ['What are the legal implications of a trust having a dated date?',
  'How does the date on a trust affect its validity and the distribution of assets?'],

In [ ]:
follow_up_queries = [["What is the background and profession of Aishwarya Rai's father?",
                      "How has Aishwarya Rai's family influenced her career in the film industry?"],
                    ['What specific information are you looking for regarding PNC routing numbers?',
                      'Are you interested in routing numbers for personal accounts, business accounts, or both?'],
                    ["In what context is the term 'mdatu' used?",
                      "Are there any specific cultural or regional references associated with 'mdatu'?"],
                    ['What specific functions or features are you looking for in a memory card utility?',
                      'Are you interested in utilities for a particular operating system or device type?'],
                    ['What specific types of medications are known to trigger lupus symptoms?',
                      'Are there particular risk factors that increase the likelihood of medication-induced lupus?'],
                    ['What are the legal implications of a trust having a dated date?',
                      'How does the date on a trust affect its validity and the distribution of assets?'],
                    ["What were the key events and outcomes of the Donner Party's journey?",
                      'How is the Donner Party relevant to the study of westward expansion in APUSH?'],
                    ['What is the best way to prepare corn on the cob before microwaving?',
                      'How long should you microwave corn on the cob for optimal results?'],
                    ['What is the history and significance of the famous statue in Brainerd, Minnesota?',
                      'Are there any notable events or festivals associated with the statue in Brainerd?'],
                    ['What is the most common mode of transportation between Columbus and Cleveland?',
                      'Are there any notable stops or attractions along the route from Columbus to Cleveland?'],
                    ['What specific aspect of Walt Disney are you interested in discussing?',
                      "Are you looking for information about Walt Disney's history, company policies, or something else?"],
                    ['What features does Windows Movie Maker offer within Windows Essentials?',
                      'Is Windows Movie Maker still supported or available for download in the latest versions of Windows?'],
                    ['What are the specific mechanisms by which lime water may aid in weight loss?',
                      'Are there any studies or evidence supporting the weight loss benefits of lime water?'],
                    ['What specific skills and knowledge are covered during Secret Service training?',
                      'Are there different training durations for various roles within the Secret Service?'],
                    ['What are the responsibilities and duties associated with the rank of E-2 in the Marines?',
                      'How does the rank of E-2 compare to other ranks in the Marine Corps in terms of pay and promotion opportunities?'],
                    ['What are the penalties for first-time offenders versus repeat offenders for this violation?',
                      'Are there any variations in ticket prices based on the state or jurisdiction?'],
                    ['What are the common medical conditions that lead to skin puckering?',
                      'How does aging contribute to the development of skin puckering?'],
                    ['What are the main features and capabilities of the Qt WebEngine browser?',
                      'How does Qt integrate with different web technologies for browser development?'],
                    ['What are the different types of cyclo compounds and their applications?',
                      'How does the structure of a cyclo compound affect its chemical properties?'],
                    ['What specific information are you looking for about Turi Cuenca, such as historical significance or current events?',
                      "Are you interested in Turi Cuenca's cultural aspects, tourist attractions, or local cuisine?"],
                    ['What is the specific address or city of Vector Space Systems?',
                      "Are there any notable landmarks or companies near Vector Space Systems' location?"],
                    ['What are the different types of hernias that a doctor might treat?',
                      'What are the common treatment options for hernias provided by specialists?'],
                    ['What are the key components that make a hypothesis testable in scientific research?',
                      'Can you provide examples of hypotheses in different fields of study?'],
                    ['What factors influence the cost of air conditioning installation?',
                      'Are there different costs associated with various types of air conditioning systems?'],
                    ['What factors contribute to the speed of gas exchange in the lungs?',
                      'How does the surface area of alveoli affect the efficiency of gas exchange?'],
                    ['What types of services are typically offered at a clinic?',
                      'How does a clinic differ from a hospital in terms of patient care?'],
                    ['What are the symptoms and characteristics of Xeroderma pigmentosum?',
                      'How is Xeroderma pigmentosum inherited and what are the genetic mutations involved?'],
                    ['What specific liver enzymes are you inquiring about, such as ALT, AST, or ALP?',
                      'Are you looking for normal ranges based on age, gender, or specific health conditions?'],
                    ['What format do account numbers and bank routing numbers typically follow on checks?',
                      'Are there specific examples of how these numbers are displayed on different types of checks?'],
                    ['What are the specific features of the water slides on these carnival ships?',
                      'Are there any age or height restrictions for using the water slides on carnival ships?'],
                    ['What types of arthritis do these doctors specialize in?',
                      'Are there any recommended clinics or hospitals in Scottsdale for arthritis treatment?'],
                    ['What were the agricultural practices used by farmers in colonial Massachusetts?',
                      'How did the climate and soil conditions in colonial Massachusetts influence crop selection?'],
                    ['What are the common symptoms of the flu that may affect the stomach?',
                      'How does the flu virus differ from other viruses that cause stomach issues, like norovirus?'],
                    ['What are the different ZIP codes for various areas within Sanford, FL?',
                      'Are there any specific neighborhoods in Sanford, FL that have unique ZIP codes?'],
                    ['What specific types of tax records should businesses keep for compliance?',
                      'Are there different retention periods for different types of businesses or tax situations?'],
                    ['What are the historical factors that influenced the development of surnames in Italy?',
                      'Can you provide examples of specific Italian surnames and their regional origins?'],
                    ['What are the best fishing techniques and gear for catching king salmon?',
                      'What are the ideal seasons and locations for king salmon fishing?'],
                    ['What sizes of Starbucks Frappuccino drinks does Costco offer and how do their prices vary?',
                      'Are there any membership discounts or promotions for purchasing Starbucks Frappuccino drinks at Costco?'],
                    ['What are the specific requirements for enrolling in the LPN program in Oklahoma?',
                      'How does the duration of the LPN program in Oklahoma compare to similar programs in other states?'],
                    ['What position does Mike Bishop currently hold that he is seeking reelection for?',
                      'What are the key issues or challenges he is facing in the upcoming election?'],
                    ['What specific types of tech classes are offered at senior centers?',
                      'Are there regional differences in the pricing of tech classes at senior centers?'],
                    ['What specific information are you looking for about Geiger in Maine, such as historical context or current events?',
                      "Are you interested in Geiger's role in a particular industry or community within Maine?"],
                    ['What are the most common symptoms associated with flower allergies?',
                      'How do flower allergy symptoms differ from other types of pollen allergies?'],
                    ['How is the median score calculated in a given dataset?',
                      'What are the implications of the median score in comparison to the mean score?'],
                    ['Can you provide examples of specific foods and the nutrients they contain?',
                      'How do nutrients function within the body compared to the role of food?'],
                    ['What activities or experiences are you looking for on the island?',
                      'Are you interested in a specific type of accommodation or amenities on the island?'],
                    ['What are the common causes of height loss in adults?',
                      'At what age does height loss typically begin to occur?'],
                    ['What factors influence the cost of cutting down a large tree, such as location and tree species?',
                      'Are there additional fees for stump removal or cleanup after the tree is cut down?'],
                    ['What are some notable characteristics of the archipelago you are referring to?',
                      'Can you specify a region or ocean where the archipelago is located?'],
                    ['What factors influence the salary of stock traders, such as experience or location?',
                      'How does the salary of stock traders compare to other finance-related professions?'],
                    ['What are the known preventive measures against hand, foot, and mouth disease?',
                      'Are there any studies linking alcohol consumption to the prevention of viral infections?'],
                    ['What are the common uses of an alias server IP in networking?',
                      'How does an alias server IP differ from a regular server IP?'],
                    ['What geological events triggered the Boxing Day tsunami in 2004?',
                      'What were the immediate and long-term impacts of the Boxing Day tsunami on affected regions?'],
                    ['How does the gamma of a covered call position compare to that of a naked call position?',
                      'What factors influence the gamma of a covered call position over time?'],
                    ['What are the primary uses of mesalamine in medical treatment?',
                      'Who developed mesalamine and what was the motivation behind its invention?'],
                    ['What is the origin and cultural significance of the name Detrick?',
                      'Are there any notable historical figures or events associated with the name Detrick?'],
                    ['What are the different types of NTP (Notice to Proceed) used in bidding processes?',
                      'How does NTP impact the timeline and execution of a project after bidding?'],
                    ['What specific tax issues has Nicolas Cage faced with the IRS?',
                      "Are there any recent updates on Nicolas Cage's financial situation or tax payments?"],
                    ['What are the most common types of immune system disorders and their symptoms?',
                      'How do environmental factors influence the development of immune system disorders?'],
                    ['What specific compounds in lemon are believed to enhance memory?',
                      'Are there any studies that support the use of lemon scent for improving memory retention?'],
                    ['How long does the paralysis from wasp venom typically last in spiders?',
                      'What species of wasps are known to paralyze spiders, and how does this process work?'],
                    ["What is the significance of the release date of 'Everybody Talks' in the context of the band's career?",
                      "Are there any notable achievements or chart positions for 'Everybody Talks' since its release?"],
                    ['What are the common symptoms that differentiate allergies from a cold?',
                      'How do the causes of allergies and colds differ in terms of triggers and pathogens?'],
                    ['What is the average temperature and precipitation in Colorado Springs during March?',
                      'Are there any significant weather events or patterns typically observed in Colorado Springs in March?'],
                    ['What specific water quality parameters are known to influence urine pH levels?',
                      'How does the source of water (e.g., tap, bottled, well) impact urine pH?'],
                    ['What specific features or functionalities does the Advent calendar offer?',
                      'Are there any particular themes or types of Advent calendars that you are interested in using?'],
                    ['What materials are best suited for framing an odd-sized poster?',
                      'Are there specific techniques for measuring and cutting a custom frame for an odd-sized poster?'],
                    ['What materials are needed to create a daylight simulation?',
                      'What are the best techniques for achieving realistic daylight effects in a room?'],
                    ['What are the different ways to use the print screen key on various operating systems?',
                      'Are there any specific applications or software that enhance the functionality of the print screen key?'],
                    ['What enzymes are involved in the process of converting ATP to ADP?',
                      'What role does this process play in cellular energy metabolism?'],
                    ['What is the age difference between Maxwell Caulfield and his wife?',
                      "What factors influenced Maxwell Caulfield's decision to marry at that time?"],
                    ['What are the tax benefits associated with using an HSA account?',
                      'How can individuals use HSA funds for qualified medical expenses?'],
                    ['What type of flight are you referring to: domestic or international?',
                      "Are you asking about a specific airline's check-in policy?"],
                    ['What are the applications of UNC threads in various industries?',
                      'How do UNC threads differ from other thread standards like UNF or metric threads?'],
                    ['What are the nutritional benefits of cape cassava?',
                      'How does cape cassava differ from other types of cassava?'],
                    ['What is the typical size range of adult pinworms?',
                      'Are there any differences in size between male and female pinworms?'],
                    ['How does the elimination period affect the overall cost of an individual disability policy?',
                      'What factors should be considered when choosing the length of the elimination period?'],
                    ['What factors can influence normal respiration levels in different individuals?',
                      'How do normal respiration levels vary across different age groups?'],
                    ['What are the physiological mechanisms by which nicotine may affect stress levels?',
                      'Are there any studies comparing the stress-relief effects of nicotine to other substances or methods?'],
                    ['What specific industries do benefit consultants typically work in?',
                      'How do benefit consultants assess and recommend employee benefits packages?']]

#Assign Respective Chunks List and Ground Truth

In [ ]:
chunks_list = [item['passages']['passage_text'] for item in train_data]
chunks_dict = [{i: text for i, text in enumerate(passages)} for passages in chunks_list]

In [ ]:
chunks_dict[0]

{0: "Pics/Satej Shinde, Shadab Khan, Bipin Kokate. Aishwarya Rai Bachchan's father Krishnaraj Rai passed away on Saturday at a suburban hospital here due to health issues. He was hospitalised a few weeks ago and was later shifted to the ICU.",
 1: "Aishwarya's father Krishnaraj Rai's funeral held, celebs pay last respects. Actress Aishwarya Rai Bachchan 's father Krishnaraj Rai passed away recently at a suburban hospital in Mumbai. Rai was hospitalised a few weeks ago and later shifted to the ICU. The funeral took place at Vile Parle Seva Sansthan Shamshan Bhoomi where Aishwarya's father-in-law Amitabh Bachchan also offered prayers.",
 2: "Aishwarya's father Krishnaraj Rai passed away at the Lilavati Hospital in suburban Bandra, the actress' publicist stated. The funeral took place around 8.30 pm at Vile Parle Seva Sansthan Shamshan Bhoomi where Aishwarya's father-in-law Amitabh Bachchan also offered prayers.",
 3: "Aishwarya's father Krishnaraj Rai's funeral: Shah Rukh Khan, Sonali Be

#Retrieval

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import euclidean
from scipy.special import expit  # Sigmoid function

def compute_normalized_distances(question, relevant_chunks):
    """
    Compute the Euclidean distance between a question and relevant chunks, then normalize using the sigmoid function.

    Args:
        question (str): The user question as a string.
        relevant_chunks (list): List of Document objects with an attribute 'page_content'.

    Returns:
        list: A list of dictionaries with chunk (page content) as key and normalized Euclidean distance as value.
    """
    # Combine question and chunks into a single list for vectorization
    all_texts = [question] + [chunk.page_content for chunk in relevant_chunks]

    # Convert texts to TF-IDF vectors
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(all_texts).toarray()

    # Compute distances
    question_vector = tfidf_matrix[0]  # Vector for the question
    chunk_vectors = tfidf_matrix[1:]   # Vectors for the chunks

    distances = [euclidean(question_vector, chunk_vector) for chunk_vector in chunk_vectors]

    # Normalize distances using the sigmoid function
    normalized_distances = expit(distances)

    # Create a list of dictionaries with chunk content and normalized distance
    result = [
        {"chunk": chunk.page_content, "distance": normalized_distance}
        for chunk, normalized_distance in zip(relevant_chunks, normalized_distances)
    ]

    return result


In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from collections import namedtuple
from langchain.schema import Document

# Instantiate the embeddings model
embeddings_model = OpenAIEmbeddings()

#adding the chunk ID
def add_chunk_id(chunk,chunks_dic):
  for id, text in chunks_dic.items():
    if text == chunk:
      return id

# Compute similarity
def compute_similarity(vector1, vector2):
    """Compute cosine similarity between two vectors."""
    return np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2))

# Loop through each question and its follow-up queries
def run_reranking(alpha,beta, gamma):
  print(len(questions))
  reranked_results = []
  for i, question in enumerate(questions):
      follow_up_query = follow_up_queries[i]
      chunks = list(chunks_dict[i].values())

      # Retrieve top relevant chunks
      vectorstore = Chroma.from_documents(documents=[Document(page_content=chunk) for chunk in chunks],
                                    embedding=OpenAIEmbeddings())
      retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})
      retrieved_chunks = retriever.get_relevant_documents(question) # list of doc objects

      # Compute normalized distances
      distance_dicts = compute_normalized_distances(question, retrieved_chunks)

      # Embed initial query and follow-up queries
      initial_query_embedding = embeddings_model.embed_query(question)
      follow_up_embeddings = [embeddings_model.embed_query(fu_query) for fu_query in follow_up_query]
      chunk_embeddings = [embeddings_model.embed_query(chunk.page_content) for chunk in retrieved_chunks]

      # Add normalized distances to metadata
      for chunk, distance_dict in zip(retrieved_chunks, distance_dicts):
          chunk.metadata["distance"] = distance_dict["distance"]

      # Compute scores
      initial_scores = [
          compute_similarity(initial_query_embedding, chunk_embedding)
          for chunk_embedding in chunk_embeddings
      ]
      follow_up_scores = [
          sum(compute_similarity(fu_embedding, chunk_embedding) for fu_embedding in follow_up_embeddings)
          for chunk_embedding in chunk_embeddings
      ]

      final_scores = [
          {
              "ID": add_chunk_id(chunk.page_content,chunks_dict[i]),
              "chunk": chunk.page_content,
              "score": alpha * initial_scores[idx] + beta * follow_up_scores[idx]- gamma * chunk.metadata["distance"]
          }
          for idx, chunk in enumerate(retrieved_chunks)
      ]

      # Sort and rerank chunks
      reranked_chunks = sorted(final_scores, key=lambda x: x["score"], reverse=True)
      reranked_results.append(reranked_chunks)
  return reranked_results

# `reranked_results` will contain the reranked chunks for each question.


/tmp/ipython-input-12-606243248.py:6: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings_model = OpenAIEmbeddings()


#Define Grounth Truths

In [ ]:
def format_reranked_results(reranked_results):
    formatted_results = {}
    for i in range(len(reranked_results)):
        question = questions[i]
        formatted_results[question] = [entry['ID'] for entry in reranked_results[i][:10]]
    return formatted_results

In [ ]:
ground_truth = [item['passages']['is_selected'] for item in train_data]
ground_truth = [[i for i, val in enumerate(inner_list) if val == 1] for inner_list in ground_truth]

In [ ]:
ground_truth

[[1],
 [],
 [],
 [],
 [5],
 [],
 [],
 [],
 [],
 [1],
 [],
 [0],
 [0],
 [8],
 [0],
 [],
 [8],
 [5],
 [0],
 [0],
 [2],
 [4],
 [],
 [1],
 [],
 [0],
 [3],
 [],
 [],
 [],
 [],
 [],
 [0, 6],
 [5],
 [8],
 [0],
 [0],
 [1],
 [2, 3],
 [],
 [],
 [7],
 [],
 [1],
 [4],
 [],
 [1],
 [7],
 [0],
 [5],
 [],
 [],
 [8],
 [],
 [],
 [],
 [7],
 [4],
 [0],
 [4],
 [],
 [1],
 [1],
 [5],
 [],
 [],
 [],
 [],
 [3],
 [],
 [],
 [7],
 [1],
 [3],
 [2],
 [0],
 [0],
 [0, 3],
 [7],
 []]

In [ ]:
relevant_chunks_dict = {question: indices for question, indices in zip(questions, ground_truth)}
relevant_chunks_dict

{'aishwarya rai father': [1],
 'routing number lookup pnc': [],
 'iwk what does mdatu meaning': [],
 'memory card utility': [],
 'medications that can trigger lupus': [5],
 'what does it mean when a trust has a dated date': [],
 'donner party apush definition': [],
 'how do you microwave corn on the cob?': [],
 'famous statue in brainerd, minnesota': [],
 'distance columbus oh to cleveland oh': [1],
 "walt disney doesn't to": [],
 'is windows movie maker in windows essentials': [0],
 'does drinking lime water help you lose weight': [0],
 'how long is secret service training': [8],
 'what is an e 2 in the marines': [0],
 'how much is a ticket for driving while operating a hand held device': [],
 'causes of skin puckering': [8],
 'what browser is qt?': [5],
 'what is a cyclo': [0],
 'turi cuenca': [0],
 'where is vector space systems located': [2],
 'what kind of doctor treats hernia': [4],
 'hypothesis is defined as a tentative explanation based on theory to predict a causal relationshi

In [ ]:
#access first pair in dict
first_key, first_value = list(relevant_chunks_dict.items())[0]
print(first_key, first_value)

aishwarya rai father [1]


#Define Evaluation Metrics

In [ ]:
from math import log2

# Evaluation Functions
def hit_rate(retrieved, relevant, k):
    retrieved_at_k = retrieved[:k]
    relevant_set = set(relevant)
    return int(any(chunk in relevant_set for chunk in retrieved_at_k))

def reciprocal_rank(retrieved, relevant):
    relevant_set = set(relevant)
    for i, chunk in enumerate(retrieved, 1):
        if chunk in relevant_set:
            return 1 / i
    return 0

def precision_at_k(retrieved, relevant, k):
    retrieved_at_k = retrieved[:k]
    relevant_set = set(relevant)
    return len(set(retrieved_at_k) & relevant_set) / len(retrieved_at_k) if retrieved_at_k else 0

def recall_at_k(retrieved, relevant, k):
    retrieved_at_k = retrieved[:k]
    relevant_set = set(relevant)
    return len(set(retrieved_at_k) & relevant_set) / len(relevant_set) if relevant_set else 0

def ndcg_at_k(retrieved, relevant, k):
    retrieved_at_k = retrieved[:k]
    relevant_set = set(relevant)
    dcg = sum((1 / log2(i + 1)) if retrieved_at_k[i - 1] in relevant_set else 0 for i in range(1, k + 1))
    idcg = sum((1 / log2(i + 1)) for i in range(1, min(len(relevant), k) + 1))
    return dcg / idcg if idcg > 0 else 0

def average_precision(retrieved, relevant):
    relevant_set = set(relevant)
    precisions = []
    for i, chunk in enumerate(retrieved, 1):
        if chunk in relevant_set:
            precisions.append(len(set(retrieved[:i]) & relevant_set) / i)
    return sum(precisions) / len(relevant_set) if relevant_set else 0

# Overall Evaluation
def evaluate_retrieval(retrieved_dict, relevant_dict, questions, k):
    results = {}
    hit_rate_scores = []
    reciprocal_ranks = []
    precision_scores = []
    recall_scores = []
    ndcg_scores = []
    average_precisions = []

    for question in questions:
        retrieved = retrieved_dict.get(question, [])
        relevant = relevant_dict.get(question, [])

        hr = hit_rate(retrieved, relevant, k)
        rr = reciprocal_rank(retrieved, relevant)
        precision = precision_at_k(retrieved, relevant, k)
        recall = recall_at_k(retrieved, relevant, k)
        ndcg = ndcg_at_k(retrieved, relevant, k)
        ap = average_precision(retrieved, relevant)

        hit_rate_scores.append(hr)
        reciprocal_ranks.append(rr)
        precision_scores.append(precision)
        recall_scores.append(recall)
        ndcg_scores.append(ndcg)
        average_precisions.append(ap)

        results[question] = {
            "hit_rate": hr,
            "reciprocal_rank": rr,
            "precision@k": precision,
            "recall@k": recall,
            "ndcg@k": ndcg,
            "average_precision": ap,
        }

    mean_hit_rate = sum(hit_rate_scores) / len(hit_rate_scores)
    mean_reciprocal_rank = sum(reciprocal_ranks) / len(reciprocal_ranks)
    mean_precision = sum(precision_scores) / len(precision_scores)
    mean_recall = sum(recall_scores) / len(recall_scores)
    mean_ndcg = sum(ndcg_scores) / len(ndcg_scores)
    mean_ap = sum(average_precisions) / len(average_precisions)

    return results, {
        "mean_hit_rate": mean_hit_rate,
        "mean_reciprocal_rank": mean_reciprocal_rank,
        "mean_precision@k": mean_precision,
        "mean_recall@k": mean_recall,
        "mean_ndcg@k": mean_ndcg,
        "mean_average_precision": mean_ap,
    }

# Example: Run Evaluation
def run_evaluation(retrieved_chunks_dict):
  k = 3
  results, summary = evaluate_retrieval(retrieved_chunks_dict, relevant_chunks_dict, questions, k)
  # Output results
  return sum(round(value, 4) for value in summary.values())

#Hyperparameter Fine Tuning

In [ ]:
#sample ranges to try
import numpy as np

# Generate sample ranges
alpha_values = np.linspace(0.5, 2, num=5)  # 5 evenly spaced values
beta_values = np.linspace(0.1, 1.5, num=5)
gamma_values = np.linspace(0.01, 1, num=5)

print("Alpha values:", alpha_values)
print("Beta values:", beta_values)
print("Gamma values:", gamma_values)

Alpha values: [0.5   0.875 1.25  1.625 2.   ]
Beta values: [0.1  0.45 0.8  1.15 1.5 ]
Gamma values: [0.01   0.2575 0.505  0.7525 1.    ]


In [ ]:
from itertools import product

def fine_tuning():
  best_result = 0
  for alpha, beta, gamma in product(alpha_values, beta_values,gamma_values):
    reranked_results = run_reranking(alpha,beta,gamma)
    retrieved_chunks_dict = format_reranked_results(reranked_results)
    evaluation_results = run_evaluation(retrieved_chunks_dict)
    if evaluation_results > best_result:
      best_result = evaluation_results
      best_params = {'alpha': alpha, 'beta': beta, 'gamma': gamma}
  return best_params

In [ ]:
print(fine_tuning())

In [ ]:
#Best parameters: #{'alpha': 0.5, 'beta': 0.1, 'gamma': 0.01}

#Post Fine Tuning Evaluation

In [ ]:
# Example: Run Evaluation
k = 2
#reranked_results = run_reranking(0.5,0.1,0.01)
reranked_results = run_reranking(1,1,1)
reranked_results

80


[[{'ID': 1,
   'chunk': "Aishwarya's father Krishnaraj Rai's funeral held, celebs pay last respects. Actress Aishwarya Rai Bachchan 's father Krishnaraj Rai passed away recently at a suburban hospital in Mumbai. Rai was hospitalised a few weeks ago and later shifted to the ICU. The funeral took place at Vile Parle Seva Sansthan Shamshan Bhoomi where Aishwarya's father-in-law Amitabh Bachchan also offered prayers.",
   'score': np.float64(1.845196977397039)},
  {'ID': 1,
   'chunk': "Aishwarya's father Krishnaraj Rai's funeral held, celebs pay last respects. Actress Aishwarya Rai Bachchan 's father Krishnaraj Rai passed away recently at a suburban hospital in Mumbai. Rai was hospitalised a few weeks ago and later shifted to the ICU. The funeral took place at Vile Parle Seva Sansthan Shamshan Bhoomi where Aishwarya's father-in-law Amitabh Bachchan also offered prayers.",
   'score': np.float64(1.845196977397039)},
  {'ID': 2,
   'chunk': "Aishwarya's father Krishnaraj Rai passed away at 

In [ ]:
len(reranked_results)

80

In [ ]:
retrieved_chunks_dict = format_reranked_results(reranked_results)
results, summary = evaluate_retrieval(retrieved_chunks_dict, relevant_chunks_dict, questions, k)

# Output results
"""for question, metrics in results.items():
    print(f"Question: {question}")
    for metric, value in metrics.items():
        print(f"  {metric}: {value:.4f}")"""
print("\nSummary:")
for metric, value in summary.items():
    print(f"{metric}: {value:.4f}")


Summary:
mean_hit_rate: 0.3000
mean_reciprocal_rank: 0.3327
mean_precision@k: 0.1500
mean_recall@k: 0.2875
mean_ndcg@k: 0.3556
mean_average_precision: 0.4346
